## Starting with the Spotify API

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [4]:
print(token)

BQB60eiBYWjbxRHS8HqrDdPCHVf551wEloTys_u1ombexV1oNeKH93n9LSRgtoKQXRlfCcnrFeifYGT8sNe4oG3UtGBw_rjguDH6WkP_Bh7MlrC2QyUUamrB2SGrdI5dZX_bAxHWN-o64mYAXk3kKw9r-Tc


#### Playlists used:
    
Created a big playlist that puts together 3 of my main playlists:
- 'Let the Chicken Chill'
- 'Indie Lusofono'
- 'Verdad'

spotify:playlist:7jIYTDhFlZoybdvGm3e6oh

- 'DISLIKED SONGS'
spotify:playlist:5zVBvIZZ4qCWDv71UZ108k

In [5]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

### Liked playlist

In [6]:
playlist_index = 0

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
like = playlists[playlist_index]['like']


In [7]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [8]:
playlist_uri

'spotify:user:Test_1:playlist:7jIYTDhFlZoybdvGm3e6oh'

In [9]:
#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

In [10]:
results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

In [11]:
results = {'items':[]}

for n in range(0,3000,100):
    new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
    results['items'] += new['items']
    
    playlist_tracks_data = results
    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    
    for track in playlist_tracks_data['items']:
        playlist_tracks_id.append(track['track']['id'])
        playlist_tracks_titles.append(track['track']['name'])
        
    #adds a list of all artists involved in the song to the list of artists for the playlist
        for artist in track['track']['artists']:
            artist_list = []
            artist_list.append(artist['name'])
        playlist_tracks_artists.append(artist_list[0])
        
    
print(len(playlist_tracks_id))

liked_df = pd.DataFrame([])
for i in range(0, len(playlist_tracks_id)):
    features = sp.audio_features(playlist_tracks_id[i])
    features_df = pd.DataFrame(features)
    liked_df = liked_df.append(features_df)

1162


In [12]:
#Merging Dataframes for getting audio features and track information

liked_df['title'] = playlist_tracks_titles
#features_df['first_artist'] = playlist_tracks_first_artists
liked_df['main_artist'] = playlist_tracks_artists
#features_df = features_df.set_index('id')
liked_df = liked_df[['id', 'title', 'main_artist',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]

liked_df['Like'] = 1
liked_df

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Like
0,0TLAptKgYxe7F0KewWH6X6,I Need A Forest Fire,Bon Iver,0.293,0.290,2,-11.596,1,0.696,0.875000,0.1000,0.0373,128.276,257220,4,1
0,2y0rHDTckWQ7of6tI1yc82,Naked Life,Oko Ebombo,0.624,0.321,0,-9.776,1,0.463,0.056000,0.1030,0.3040,73.996,255030,4,1
0,5nVE45ithgyVocZp5XLAk6,The Bug Collector,Haley Heynderickx,0.538,0.419,10,-9.705,1,0.777,0.329000,0.0967,0.5290,99.622,228460,4,1
0,66VuLX5s1nrZzzQqJqPbri,Simply Are,Arto Lindsay,0.682,0.694,7,-11.393,0,0.289,0.714000,0.0961,0.9540,173.851,224107,4,1
0,0X61Ud5Uzv44RvY3s1SsuO,Imogen,Nick Mulvey,0.493,0.584,5,-6.894,1,0.850,0.000353,0.1140,0.2340,86.909,255533,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,7qH9Z4dJEN0l9bidizW7fq,Anchor,Novo Amor,0.457,0.408,9,-11.474,1,0.803,0.884000,0.1260,0.1280,117.052,257533,4,1
0,1sgBtdd3OgBABV9L5tM8K8,Yi,Bon Iver,0.661,0.407,10,-21.583,0,0.730,0.000000,0.8620,0.2770,78.382,31547,4,1
0,4qyfir5Yr7nfo05g6cyFMT,Promise,Ben Howard,0.367,0.189,2,-15.736,1,0.889,0.428000,0.3640,0.1770,156.643,384080,4,1
0,3CAX47TnPqTujLIQTw8nwI,Old Pine,Ben Howard,0.401,0.364,5,-10.836,1,0.450,0.050300,0.1620,0.2240,129.570,328507,4,1


### Disliked playlist

In [13]:
playlist_index = 1

playlists = json.load(open('playlists_like_dislike.json'))
playlist_uri = playlists[playlist_index]['uri']
dislike = playlists[playlist_index]['like']

#Getting Information of tracks in Spotify playlist

uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist_tracks(username, playlist_id,  offset = 0)

In [14]:
playlist_id

'4cth4hjHHeCQmOUsqdZFTU'

In [15]:
results = {'items':[]}

for n in range(0,3000,100):
    new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
    results['items'] += new['items']
    
    playlist_tracks_data = results
    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    
    for track in playlist_tracks_data['items']:
        playlist_tracks_id.append(track['track']['id'])
        playlist_tracks_titles.append(track['track']['name'])
        
    #adds a list of all artists involved in the song to the list of artists for the playlist
        for artist in track['track']['artists']:
            artist_list = []
            artist_list.append(artist['name'])
        playlist_tracks_artists.append(artist_list[0])
        
    
print(len(playlist_tracks_id))

disliked_df = pd.DataFrame([])
for i in range(0, len(playlist_tracks_id)):
    features = sp.audio_features(playlist_tracks_id[i])
    features_df = pd.DataFrame(features)
    disliked_df = disliked_df.append(features_df)

1351


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [16]:
#Merging Dataframes for getting audio features and track information

disliked_df['title'] = playlist_tracks_titles
#features_df['first_artist'] = playlist_tracks_first_artists
disliked_df['main_artist'] = playlist_tracks_artists
#features_df = features_df.set_index('id')
disliked_df = disliked_df[['id', 'title', 'main_artist',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]
disliked_df['Like'] = 0
disliked_df

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Like
0,2VMMhJxWI2aipSBTkpqXjI,BPM 170,MC Kevin o Chris,0.710,0.504,6.0,-5.212,1.0,0.60500,0.003210,0.0880,0.8950,84.318,203520.0,4.0,0
0,0K6ldKbWkd91UJS3MlTRyR,Largadão,Lexa,0.888,0.920,11.0,-1.673,1.0,0.35800,0.001820,0.0666,0.7640,93.987,132766.0,4.0,0
0,25gD4PQjxCLnwIY2n23aEl,Flores,Luísa Sonza,0.626,0.599,6.0,-5.869,0.0,0.25500,0.000000,0.1600,0.6210,100.004,155798.0,4.0,0
0,4eqpOs8YgVHLJPnBPD41zN,Mete Dança,Ruxell,0.754,0.928,1.0,-2.120,1.0,0.28100,0.000080,0.1440,0.8180,99.746,122400.0,4.0,0
0,7jV67Bz9qRkk57FMedir8S,Nosso Relacionamento,Dallass,0.568,0.471,8.0,-8.769,0.0,0.44300,0.000000,0.0929,0.4550,134.817,200769.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,7szuecWAPwGoV1e5vGu8tl,In Your Eyes,The Weeknd,0.667,0.719,7.0,-5.371,0.0,0.00285,0.000081,0.0736,0.7170,100.021,237520.0,4.0,0
0,3eekarcy7kvN4yt5ZFzltW,HIGHEST IN THE ROOM,Travis Scott,0.598,0.427,7.0,-8.764,0.0,0.05460,0.000006,0.2100,0.0605,76.469,175721.0,4.0,0
0,4Ty4XfOHdUFIXmVtBIFMIG,XPIDI,benji price,0.840,0.475,5.0,-6.665,0.0,0.64100,0.000000,0.0983,0.8820,77.006,172064.0,4.0,0
0,1xQ6trAsedVPCdbtDAmk0c,Savage Love (Laxed - Siren Beat),Jason Derulo,0.767,0.481,0.0,-8.520,0.0,0.23400,0.000000,0.2690,0.7610,150.076,171375.0,4.0,0


### Final dataframe

In [17]:
frames = [liked_df, disliked_df]

my_data = pd.concat(frames)

In [18]:
my_data

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Like
0,0TLAptKgYxe7F0KewWH6X6,I Need A Forest Fire,Bon Iver,0.293,0.290,2.0,-11.596,1.0,0.69600,0.875000,0.1000,0.0373,128.276,257220.0,4.0,1
0,2y0rHDTckWQ7of6tI1yc82,Naked Life,Oko Ebombo,0.624,0.321,0.0,-9.776,1.0,0.46300,0.056000,0.1030,0.3040,73.996,255030.0,4.0,1
0,5nVE45ithgyVocZp5XLAk6,The Bug Collector,Haley Heynderickx,0.538,0.419,10.0,-9.705,1.0,0.77700,0.329000,0.0967,0.5290,99.622,228460.0,4.0,1
0,66VuLX5s1nrZzzQqJqPbri,Simply Are,Arto Lindsay,0.682,0.694,7.0,-11.393,0.0,0.28900,0.714000,0.0961,0.9540,173.851,224107.0,4.0,1
0,0X61Ud5Uzv44RvY3s1SsuO,Imogen,Nick Mulvey,0.493,0.584,5.0,-6.894,1.0,0.85000,0.000353,0.1140,0.2340,86.909,255533.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,7szuecWAPwGoV1e5vGu8tl,In Your Eyes,The Weeknd,0.667,0.719,7.0,-5.371,0.0,0.00285,0.000081,0.0736,0.7170,100.021,237520.0,4.0,0
0,3eekarcy7kvN4yt5ZFzltW,HIGHEST IN THE ROOM,Travis Scott,0.598,0.427,7.0,-8.764,0.0,0.05460,0.000006,0.2100,0.0605,76.469,175721.0,4.0,0
0,4Ty4XfOHdUFIXmVtBIFMIG,XPIDI,benji price,0.840,0.475,5.0,-6.665,0.0,0.64100,0.000000,0.0983,0.8820,77.006,172064.0,4.0,0
0,1xQ6trAsedVPCdbtDAmk0c,Savage Love (Laxed - Siren Beat),Jason Derulo,0.767,0.481,0.0,-8.520,0.0,0.23400,0.000000,0.2690,0.7610,150.076,171375.0,4.0,0


In [19]:
#export csv

my_data.to_csv (r'my_data.csv', index = False, header=True)

In [20]:
#plt.figure(figsize=(20,30))
#sns.countplot(features_df['main_artist'])
#plt.xticks(rotation=90)